In [14]:
%pip install ultralytics

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import ultralytics
ultralytics.__version__

'8.2.60'

In [16]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import*

model=YOLO('yolov8s.pt')

In [17]:
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'bus', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [18]:
tracker=Tracker()
count=0

In [19]:
cap=cv2.VideoCapture('highway.mp4')

In [20]:
down={}
up={}

counter_down=[]
counter_up=[]

In [21]:
while True:    
    ret,frame = cap.read()
    if not ret:
        break
    count += 1
    frame=cv2.resize(frame,(1020,500))
   

    results=model.predict(frame)
 #   print(results)
    a=results[0].boxes.data
    a = a.detach().cpu().numpy() 
    px=pd.DataFrame(a).astype("float")
    #print(px)

    list=[]
             
    for index,row in px.iterrows():
#        print(row) 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if 'car' in c:
            list.append([x1,y1,x2,y2])
            #print(c)

    bbox_id=tracker.update(list)
    #print(bbox_id)
    for bbox in bbox_id:
        x3,y3,x4,y4,id=bbox
        cx=int(x3+x4)//2
        cy=int(y3+y4)//2
        cv2.circle(frame,(cx,cy),4,(0,0,255),-1) #draw ceter points of bounding box
        cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)  # Draw bounding box
        cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)


        red_line_y=198
        blue_line_y=268   
        offset = 7
        
  
        if red_line_y < (cy + offset) and red_line_y > (cy - offset):
          down[id]=cy   
        if id in down:
           if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):         
             cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
             cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
             #counter+=1
             counter_down.append(id)  

        # condition for cars entering from  blue line
        if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
          up[id]=cy   
        if id in up:
           if red_line_y < (cy + offset) and red_line_y > (cy - offset):         
             cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
             cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
            
             counter_up.append(id) 




    
    text_color = (255,255,255) 
    red_color = (0, 0, 255)  # (B, G, R)   
    blue_color = (255, 0, 0)  # (B, G, R)
    green_color = (0, 255, 0)  # (B, G, R)  

    cv2.line(frame,(172,198),(774,198),red_color,3)  
    cv2.putText(frame,('red line'),(172,198),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    
    cv2.line(frame,(8,268),(927,268),blue_color,3)  
    cv2.putText(frame,('blue line'),(8,268),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)    


    downwards = (len(counter_down))
    cv2.putText(frame,('going down - ')+ str(downwards),(60,40),cv2.FONT_HERSHEY_SIMPLEX, 0.5, green_color, 1, cv2.LINE_AA)    

    
    upwards = (len(counter_up))
    cv2.putText(frame,('going up - ')+ str(upwards),(60,60),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)  

    cv2.imshow("frames", frame)
    if cv2.waitKey(1)&0xFF==27:
        break
cap.release()
cv2.destroyAllWindows()


0: 320x640 8 cars, 2 trucks, 883.8ms
Speed: 0.0ms preprocess, 883.8ms inference, 6.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 893.5ms
Speed: 2.8ms preprocess, 893.5ms inference, 16.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 cars, 2 trucks, 910.2ms
Speed: 0.0ms preprocess, 910.2ms inference, 7.8ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 cars, 2 trucks, 953.8ms
Speed: 9.1ms preprocess, 953.8ms inference, 16.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 946.0ms
Speed: 10.8ms preprocess, 946.0ms inference, 6.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 863.1ms
Speed: 4.9ms preprocess, 863.1ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 881.9ms
Speed: 4.8ms preprocess, 881.9ms inference, 14.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 913.8ms
Speed:

KeyboardInterrupt: 